In [1]:
##impor every thing that are need in this project 
import os 
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import json
import pandas as pd
import traceback 
import PyPDF2


/home/sahil/Question_Answ/question/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## in this project i use gemini 

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="")

In [3]:
## that are the format of response 

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [4]:
## this template 1 
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

""" 
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [5]:
## create the prompt 

quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
) 

## now create a chain for prompt one 

llm_chain1=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz")

/home/sahil/Question_Answ/question/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [6]:
## create prompt for second template 

quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
) 

review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review") 

In [7]:
## now use squentail chain 

generate_evaluate_chain=SequentialChain(chains=[llm_chain1, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"])

In [10]:
from PyPDF2 import PdfReader
## now test our model 
# file_path="/home/sahil/Question_Answ/data.txt"
# with open(file_path, 'r') as file:
#     TEXT = file.read() 
def read_file(file_path):
    if file_path.endswith(".pdf"):
        with open(file_path, "rb") as file:
            pdf_reader = PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            return text

In [15]:
path="/home/sahil/Question_Answ/yolo.pdf" 
text=read_file(path) 
# print(text)

In [16]:
NUMBER=10
SUBJECT="CNN"
TONE="simple"

In [17]:
response=generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/home/sahil/Question_Answ/question/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [18]:
quiz=response.get("quiz")

In [19]:
quiz=quiz.split("### RESPONSE_JSON\n")[1]
quiz

'{"1": {"mcq": "What is the main idea behind the YOLO object detection system?", "options": {"a": "Using a single convolutional network to predict bounding boxes and class probabilities directly from images.", "b": "Repurposing classifiers to perform object detection.", "c": "Using region proposal methods to generate potential bounding boxes.", "d": "Employing a sliding window approach to evaluate classifiers at various locations."}, "correct": "a"}, "2": {"mcq": "How does YOLO reason globally about the image?", "options": {"a": "By dividing the image into a grid and having each grid cell predict bounding boxes and class probabilities.", "b": "By using a sliding window approach to evaluate the entire image.", "c": "By generating region proposals and running a classifier on each proposed box.", "d": "By using a complex pipeline of separate components."}, "correct": "a"}, "3": {"mcq": "What are the benefits of YOLO\'s unified architecture?", "options": {"a": "It is extremely fast and can

In [20]:
quiz=json.loads(quiz)

In [21]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [22]:
quiz_table_data

[{'MCQ': 'What is the main idea behind the YOLO object detection system?',
  'Choices': 'a: Using a single convolutional network to predict bounding boxes and class probabilities directly from images. | b: Repurposing classifiers to perform object detection. | c: Using region proposal methods to generate potential bounding boxes. | d: Employing a sliding window approach to evaluate classifiers at various locations.',
  'Correct': 'a'},
 {'MCQ': 'How does YOLO reason globally about the image?',
  'Choices': 'a: By dividing the image into a grid and having each grid cell predict bounding boxes and class probabilities. | b: By using a sliding window approach to evaluate the entire image. | c: By generating region proposals and running a classifier on each proposed box. | d: By using a complex pipeline of separate components.',
  'Correct': 'a'},
 {'MCQ': "What are the benefits of YOLO's unified architecture?",
  'Choices': 'a: It is extremely fast and can process images in real-time. | b:

In [23]:
quiz=pd.DataFrame(quiz_table_data)

In [24]:
quiz.to_csv("ml5.csv",index=False)